# < 알파벳을 기준으로 그룹별로 정렬하기 >
    * 전달받은 데이터는 외국인 셀럽들의 데이터이다. 회사에서 요구한 것은 각 셀럽들의 성이 아닌 이름의 맨 앞 알파벳을 기준으로 그룹을 지어달라는 것!
예 : Amanda Seyfried 는 Actor 그룹의  'S'그룹

1) 성이 아닌 이름을 기준으로 할 것
    - 예외 케이스 고려하기 ( 미들네임 or '.'이 이름에 있는 경우 등)

2) 정렬 기준은 그룹 -> 이름의 알파벳. 각각 인물들의 직업이 우선순위로 그룹화 되어있어야 함



In [2]:
import pandas as pd
import numpy as np
import re

In [10]:
# 전달받은 data set
data = pd.read_excel("/Users/myhellebs/Downloads/celeb_data.xlsx") 
data

,pk_num,keyword_name,kl_name_en,type
0,1,Gael García Bernal,Gael García Bernal,Actor
1,2,Robert De Niro,Robert De Niro,Actor
2,3,Mark Boone Junior,Mark Boone Junior,Actor
3,4,Callum Keith Rennie,Callum Keith Rennie,Actor
4,5,Edward James Olmos,Edward James Olmos,Actor
...,...,...,...,...
88,89,Damon Wayans Jr.,Damon Wayans Jr.,Actor
89,90,O'Shea Jackson Jr.,O'Shea Jackson Jr.,Actor
90,91,Jorge Lendeborg Jr.,Jorge Lendeborg Jr.,Actor
91,92,"George Tillman, Jr.","George Tillman, Jr.",Director : Male


## < 기준 및 예외 케이스 정하기 >
    - 성 + 이름(두개의 단어)으로 되어 있는 사람은 이름의 맨 앞글자 대문자만 추출
    - 미들네임이 있는 경우에는 맨 뒤에 있는 이름으로 선택. 예)Jennifer Yuh Nelson = N그룹
    - 이름에 Junior or Jr가 있는 경우에는, jr바로 이전 단어를 이름으로 선택. 예)John Gallagher Jr. = G그룹
    - 이름에 특수문자('.','-' etc)가 있는건 특이 케이스 이므로 주의해서 따로 확인할 것

In [89]:
global check_list
check_list = []

def Upper_Group(x):
    
    unit = x.split(" ")
    Alphabet = re.compile('[A-Z]') # 대문자만 추출  
    sc = re.compile("[^a-zA-Z\s]") # 공백을 제외한 특수문자가 있는 이름들의 확인을 위한 정규식
    
    upper = Alphabet.findall(x)
    
    try :  
        if ('Jr.' in unit) or ('Junior' in unit):
            return upper[-2]
        
        else : # stereo type의 모든 이름(성 + 이름)
            return upper[-1] 
        
    finally :     
        if sc.findall(x): # 특수문자가 있으면
            check_list.append(x)    


In [99]:
# 이름의 대문자를 추출한 새로운 칼럼 생성
data['upper'] = data['keyword_name'].apply(Upper_Group)
data

,pk_num,keyword_name,kl_name_en,type,upper
0,1,Gael García Bernal,Gael García Bernal,Actor,B
1,2,Robert De Niro,Robert De Niro,Actor,N
2,3,Mark Boone Junior,Mark Boone Junior,Actor,B
3,4,Callum Keith Rennie,Callum Keith Rennie,Actor,R
4,5,Edward James Olmos,Edward James Olmos,Actor,O
...,...,...,...,...,...
88,89,Damon Wayans Jr.,Damon Wayans Jr.,Actor,W
89,90,O'Shea Jackson Jr.,O'Shea Jackson Jr.,Actor,J
90,91,Jorge Lendeborg Jr.,Jorge Lendeborg Jr.,Actor,L
91,92,"George Tillman, Jr.","George Tillman, Jr.",Director : Male,T


In [104]:
# 특수문자가 있던 이름들을 조회
data[data['keyword_name'].isin(check_list)].iloc[40:50]

,pk_num,keyword_name,kl_name_en,type,upper
49,50,Malcolm D. Lee,Malcolm D. Lee,Director : Male,L
50,51,Jon S. Baird,Jon S. Baird,Director : Male,B
51,52,Christian E. Christiansen,Christian E. Christiansen,Director : Male,C
52,53,S.S. Rajamouli,S.S. Rajamouli,Director : Male,R
53,54,Michaël R. Roskam,Michaël R. Roskam,Director : Male,R
54,55,J. C. Chandor,J. C. Chandor,Director : Male,C
55,56,Rob W. King,Rob W. King,Director : Male,K
56,57,S. Craig Zahler,S. Craig Zahler,Director : Male,Z
66,67,Stephen St. Leger,Stephen St. Leger,Director : Male,L
72,73,Louis C.K.,Louis C.K.,Actor,L


In [105]:
# pk_num = 73, name = 'Louis C.K.' 특이 데이터 확인 후 수정하기
data.loc[data.pk_num == 73, 'upper'] = "L"

# 특이 데이터의 알파벳 그룹이 수정됐음을 확인
data[data['keyword_name'].isin(check_list)].iloc[47:50]

,pk_num,keyword_name,kl_name_en,type,upper
56,57,S. Craig Zahler,S. Craig Zahler,Director : Male,Z
66,67,Stephen St. Leger,Stephen St. Leger,Director : Male,L
72,73,Louis C.K.,Louis C.K.,Actor,L


In [132]:
alphabet_group = list(data['upper'].unique())
alphabet_group = sorted(alphabet_group)
type_group = list(data['type'].unique())

In [138]:
final = []

for job in type_group :
    
    for alpha in alphabet_group :
        
        pk_list = list(data[(data['type'] == job) & (data['upper'] == alpha)]['pk_num'])
        pk_list = str(pk_list).replace("[","").replace("]","")
        
        names = list(data[(data['type'] == job) & (data['upper'] == alpha)]['keyword_name'])
        names = str(names).replace("[","").replace("]","").replace("\'","")
        
        result = {'type':job ,'upper':alpha, 'pk_num':pk_list, 'celeb':names}
        
        final.append(result)
        

In [139]:
# 최종 결과
pd.DataFrame(final)#.sort_values('upper')

,type,upper,pk_num,celeb
0,Actor,A,,
1,Actor,B,"1, 3, 74","Gael García Bernal, Mark Boone Junior, Mark Bo..."
2,Actor,C,"76, 79, 83","Clifton Collins Jr., John Cothran, Jr., Harry ..."
3,Actor,D,75,Robert Downey Jr.
4,Actor,E,,
...,...,...,...,...
61,Director : Male,S,"19, 22, 25, 39, 41, 58","David F. Sandberg, M. Night Shyamalan, Frankli..."
62,Director : Male,T,"32, 70, 92","J. Lee Thompson, Jonathan van Tulleken, George..."
63,Director : Male,V,,
64,Director : Male,W,"23, 24","Fred M. Wilcox, David S. Ward"
